In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
from experiments.run import handle_combine_modes

logging.basicConfig(level=logging.INFO)

In [ ]:
experiment_folder = "."
experiment_name = "combine_modes_results/recv_data_no_sparse_False_recv_mod_add_data_backward_True_make_new_opt_True"
experiment_name = "combine_modes_results/gt_recv_data_no_sparse_False_recv_mod_add_data_backward_True_make_new_opt_True"
dataset = "kmnist"
algo = "modular"

use_contrastive = False
num_trains_per_class = 64 if dataset != "cifar100" else 256
seed = 0
parallel = False
task_id = 3


agent_id = 0

In [ ]:
def modify_cfg(net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg):
    cfg = handle_combine_modes(cfg)
    return  net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg

In [ ]:
save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)

fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
# fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
fleet.change_save_dir('debug_combine_modes_results')

In [ ]:
agent = fleet.agents[agent_id]
if 'gt' not in experiment_name:
    comm = agent.communicator['recv_data']
    print(comm.neighbors)

In [ ]:
if 'gt' not in experiment_name:
    fleet.communicate(task_id, end_epoch=None, comm_freq=None, num_epochs=None, strategy='recv_data')
else:
    fleet.communicate(task_id, end_epoch=None, comm_freq=None, num_epochs=None)

In [ ]:
if 'gt' not in experiment_name:
    print(comm.incoming_data)
else:
    print(agent.incoming_data)

In [ ]:
agent.eval_test(task_id=task_id, include_avg=True)

In [ ]:
agent.incoming_data